# Notes 2023-03-03

## Decorators

As seen before

    @pytest.mark.parametrize
    def test_xxx()
        ....
        
example of decorator

A decorator is a function of a function returning a function

In [10]:
def f(x):
    return x*x

In [11]:
def g(func):
    print('Hello')
    return func

In [12]:
myfunc = g(f)

Hello


In [13]:
f(2)

4

In [14]:
myfunc(2)

4

In [18]:
def g(func):
    def wrapper(x):
        print('Hello')
        return func(x)
    return wrapper

In [19]:
myfunc = g(f)

In [20]:
myfunc(2)

Hello


4

New notation

In [21]:
@g
def f(x):
    return x*x

Equivalent to 

    f = g(f)

In [22]:
f(2)

Hello


4

### A more useful example

In [23]:
import time
def slow(x):
    result = x * x
    time.sleep(1)
    return result

In [28]:
slow.__name__

'slow'

In [25]:
slow(2)

4

In [27]:
# Take the time bfore and after and calculate difference
t1 = time.time()
slow(2)
t2 = time.time()
print("Time spend in slow", t2 - t1)

Time spend in slow 1.000981092453003


In [36]:
def time_me(f):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = f(*args, **kwargs)
        t2 = time.time()
        print(f"Time spent in {f.__name__}", t2 - t1)
        return result
    return wrapper
        

In [37]:
@time_me
def slow(x, c=1.0):
    result = c * x * x
    time.sleep(1)
    return result

In [38]:
slow(2)

Time spent in slow 1.0010461807250977


4.0

In [39]:
slow(3)

Time spent in slow 1.0010802745819092


9.0

In [40]:
slow(3, c=2)

Time spent in slow 1.0011065006256104


18

## another example: track input/output for a function

In [41]:
def say(greet, name):
    return greet + "," +  name

In [42]:
say("Hello", 'Joe')

'Hello,Joe'

In [44]:
def trace(f):
    def wrapper(*args, **kwargs):
        print(f.__name__, 'called with', args, kwargs)
        result = f(*args, **kwargs)
        print(f.__name__, 'returns', result)
        return result
    return wrapper

In [45]:
@trace
def say(greet, name):
    return greet + "," +  name

In [46]:
say("Hello", 'Joe')

say called with ('Hello', 'Joe') {}
say returns Hello,Joe


'Hello,Joe'

In [48]:
@time_me
@trace
def say(greet, name):
    return greet + "," +  name

In [49]:
say("Hello", 'Joe')

say called with ('Hello', 'Joe') {}
say returns Hello,Joe
Time spent in wrapper 3.4809112548828125e-05


'Hello,Joe'

Note the 
    
<code>Time spent in <span style="color: red;">wrapper</red></code>

output. Not useful information - `functools.wraps` to the rescue

In [51]:
from functools import wraps

def trace(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        print(f.__name__, 'called with', args, kwargs)
        result = f(*args, **kwargs)
        print(f.__name__, 'returns', result)
        return result
    return wrapper

@time_me
@trace
def say(greet, name):
    return greet + "," +  name

say("Hello", 'Joe')

say called with ('Hello', 'Joe') {}
say returns Hello,Joe
Time spent in say 0.00018644332885742188


'Hello,Joe'

The helper decorator functools.wraps keeps naming consistent

## Context managers

classes that support `with` statements. example

example

    with open(filename, 'w') as f:
        f.write()

In [53]:
class TimeMe:
    ...

In [54]:
with TimeMe() as tm:
    pass

AttributeError: __enter__

In [62]:
class TimeMe:
    def __enter__(self):
        self.t1 = time.time()
    def __exit__(self, *args):
        self.t2 = time.time()
        print("Time spent:", self.t2 - self.t1)

In [64]:
def slow(x, c=1.0):
    result = c * x * x
    time.sleep(1)
    return result

In [66]:
with TimeMe():
    print(slow(2, c=2))

8
Time spent: 1.0013630390167236


## Iterators

In [67]:
# strings, lists, dictionaries

In [68]:
for c in 'abc':
    print(c)

a
b
c


In [69]:
for element in [1,2,3]:
    print(element)

1
2
3


In [70]:
for i in range(3):
    print(i)

0
1
2


In [78]:
class MyRange:
    def __init__(self, limit):
        self.limit = limit
        self.counter = -1
        
    def __iter__(self):  # called in the beginning of a for loop
        return self
    
    def __next__(self):  # return values during iteraion
        self.counter += 1
        if self.counter == self.limit:
            raise StopIteration
        time.sleep(1)
        return self.counter
    
for i in MyRange(3):
    print(i)

0
1
2


In [80]:
my_range = MyRange(3)

In [81]:
my_range.limit

3

In [82]:
my_range.counter

-1

In [83]:
my_range_iterator = iter(my_range)

In [84]:
next(my_range_iterator)

0

In [85]:
next(my_range_iterator)

1

In [86]:
next(my_range_iterator)

2

In [87]:
next(my_range_iterator)

StopIteration: 

In [88]:
next(my_range_iterator)

4

 oops unexpecteced behaviour, implementation must be improved